In [105]:
import pandas as pd
import numpy as np 
import plotly.express as ex 
import plotly.graph_objects as go

# 数据清洗&预处理

In [106]:
#加载三张数据集
user_df = pd.read_csv('databases/user—test.csv',encoding='gb18030')
order_df = pd.read_csv('databases/Order-test.csv')
consult_df = pd.read_csv('databases/consult-test.csv')

## 处理 User 表格

In [107]:
print(user_df.info(),"\n")
missingValue_stat()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 870460 entries, 0 to 870459
Data columns (total 11 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   user_id         870460 non-null  int64  
 1   sex             62753 non-null   object 
 2   age             61970 non-null   float64
 3   country         866918 non-null  object 
 4   province_name   866918 non-null  object 
 5   city_name       866918 non-null  object 
 6   city_level      863913 non-null  object 
 7   is_member_flag  870460 non-null  int64  
 8   first_day       811277 non-null  object 
 9   last_day        870460 non-null  object 
 10  be_member_time  596011 non-null  object 
dtypes: float64(1), int64(2), object(8)
memory usage: 73.1+ MB
None 

Each Column Missing Value
user_id : 0
sex : 807707
age : 808490
country : 3542
province_name : 3542
city_name : 3542
city_level : 6547
is_member_flag : 0
first_day : 59183
last_day : 0
be_member_time : 274449


In [108]:
# 这里 sex & age Missing value 是80w+, 这个数据集总共才87W, 缺失值总共约占数据集92%
# 直接去掉这两行
user_df = user_df.drop(['sex','age'],axis=1)

In [109]:
# 查看province_name 为缺失值有3542个，由于省名缺失，对后续分析有影响。
# 所以,直接将这三千多行数据去掉，约占总数据集的0.4%，影响不大
# user_df = user_df.dropna(subset=['province_name'])
# columnlist = user_df.columns.values
print(user_df.isna().sum())
print(user_df.shape)

user_id                0
country             3542
province_name       3542
city_name           3542
city_level          6547
is_member_flag         0
first_day          59183
last_day               0
be_member_time    274449
dtype: int64
(870460, 9)


In [110]:
#缺失city_level这些地方属于一些特殊的省直管县级市，按照这种方法，使用‘其他城市’去替换Nan
user_df['city_level'].fillna("其他城市",inplace=True)
columnlist = user_df.columns.values
missingValue_stat()

Each Column Missing Value
user_id : 0
3542try : 
province_name : 3542
city_name : 3542
city_level : 0
is_member_flag : 0
first_day : 59183
last_day : 0
be_member_time : 274449


/var/folders/4q/j0ysgpnj36545bb_2_dt0tr80000gn/T/ipykernel_65226/1500180173.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  user_df['city_level'].fillna("其他城市",inplace=True)


In [111]:
#关于填补first_day 的策略，我分下面两种情况进行填补
# 1. 如果开通会员的话，be_member_time就是他的first_time
# 2. 如果没有开通会员的话，first_time 就是它的last_day, 把这类临时用户
user_df['first_day'].fillna(user_df['be_member_time'],inplace=True)
user_df.reset_index(drop=True)
user_df.isnull().sum()

/var/folders/4q/j0ysgpnj36545bb_2_dt0tr80000gn/T/ipykernel_65226/2803777748.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  user_df['first_day'].fillna(user_df['be_member_time'],inplace=True)


user_id                0
country             3542
province_name       3542
city_name           3542
city_level             0
is_member_flag         0
first_day          19480
last_day               0
be_member_time    274449
dtype: int64

In [112]:
#检测是否有重复row
user_df.duplicated().sum()

np.int64(0)

In [113]:
user_df['first_day'] = pd.to_datetime(user_df['first_day'],errors='coerce')
user_df['last_day'] = pd.to_datetime(user_df['last_day'],errors='coerce')
user_df['be_member_time'] = pd.to_datetime(user_df['be_member_time'],errors='coerce')

condition1 = user_df['first_day'] > user_df['be_member_time']
user_df = user_df[~condition1]

condition2 = user_df['first_day'] > user_df['last_day']
user_df = user_df[~condition2]

condition3 = user_df['last_day'] < user_df['be_member_time']
user_df = user_df[~condition3]

user_df.shape

(562272, 9)

In [114]:
user_df['renew_time'] = (user_df['be_member_time'] + pd.DateOffset(days=365))
user_df['is_renew'] = (user_df['renew_time'] <= user_df['last_day']).astype(int)

In [117]:
user_df

,user_id,country,province_name,city_name,city_level,is_member_flag,first_day,last_day,be_member_time,renew_time,is_renew
0,570009353210116,中国,山东,潍坊,三线城市,1,2023-04-03,2023-08-02,2023-04-03 19:42:00,2024-04-02 19:42:00,0
9,570023426180132,中国,北京,北京,一线城市,0,2023-03-04,2023-03-09,NaT,NaT,0
12,570025239650135,中国,上海,上海,一线城市,1,2023-03-08,2023-08-17,2023-07-15 18:25:00,2024-07-14 18:25:00,0
17,570026463920104,中国,黑龙江,哈尔滨,二线城市,1,NaT,2023-07-18,2022-08-16 14:37:00,2023-08-16 14:37:00,0
18,570027668630107,中国,四川,成都,新一线城市,0,2023-03-25,2023-03-25,NaT,NaT,0
...,...,...,...,...,...,...,...,...,...,...,...
870452,570072268180146,中国,湖南,长沙,新一线城市,0,2023-06-30,2023-06-30,NaT,NaT,0
870453,570072163840123,中国,天津,天津,新一线城市,0,2023-06-29,2023-06-29,NaT,NaT,0
870455,570072597650156,中国,重庆,重庆,新一线城市,0,2023-06-30,2023-06-30,NaT,NaT,0
870456,570072523900118,中国,山东,潍坊,三线城市,0,2023-06-30,2023-08-03,NaT,NaT,0


In [121]:
user_df[user_df['user_id'] == 570072529930156]

,user_id,country,province_name,city_name,city_level,is_member_flag,first_day,last_day,be_member_time,renew_time,is_renew
870459,570072529930156,中国,湖北,黄冈,四线城市,0,2023-06-30,2023-06-30,NaT,NaT,0
